In [1]:
import pandas as pd

# Define the file path
nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
bridges_selected_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_bridges_sel = pd.read_csv(
    bridges_selected_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_17860\232746929.py:12: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_17860\232746929.py:13: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [4]:
import pandas as pd

# Define the columns that should be converted to numeric data types
numeric_columns = ['Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
                   'Počet_otvorov', 'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m',
                   'Volná_šírka_mosta', 'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
                   'Plocha_mosta_m2', 'n_2012', 'n_2013', 'n_2014', 'n_2015', 'n_2016',
                   'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021', 'n_2022', 'n_2023']

# Convert the selected columns to numeric data types (integers or floats)
for col in numeric_columns:
    df_bridges_all[col] = pd.to_numeric(df_bridges_all[col], errors='coerce')

In [5]:
import geopandas as gpd
# Replace the file path with your actual file path
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Nitra\regions_epsg_4326.geojson'
# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame
# gdf.head()

In [6]:
df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [49]:
filtered_bridges = df_bridges_all[
    (df_bridges_all["Kraj"] == "Košice") &
    (df_bridges_all["n_2023"] >= 6) &
    (df_bridges_all["Material"] == "prefabrikovaný predpätý betón") &
    ((df_bridges_all["Trieda_PK"] == "cesta II. triedy") |
     (df_bridges_all["Trieda_PK"] == "cesta III. triedy"))
]

In [50]:
filtered_bridges["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy    30
cesta II. triedy     15
Name: count, dtype: int64

In [51]:
import folium


def plot_filtered_bridges(df, gdf):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng.svg"

    # Add a marker for each bridge
    for index, row in df.iterrows():
        ic_size = 50
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{row['Trieda_PK']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

        # Filter the GeoDataFrame to include only the Nitra Region
    nitra_region_gdf = gdf[gdf['NM4'] == 'Košický']

    # Define custom style for the Nitra Region polygon
    def style_function(x): return {
        'fillColor': '#FF6600', 'color': '#FF6600', 'weight': 2}

    # Add the boundaries of the Nitra Region to the map with the custom style
    folium.GeoJson(nitra_region_gdf,
                   name='Nitra Region Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame)
map = plot_filtered_bridges(filtered_bridges, gdf)
map.save('filtered_bridges.html')
map

In [52]:
df_bridges_all["Kraj"].value_counts()

Kraj
Prešov             1851
Banská Bystrica    1633
Žilina             1376
Trenčín             967
Košice              801
Nitra               765
Trnava              534
Bratislava          382
Nezistený           193
Name: count, dtype: int64

In [62]:
filtered_bridges2 = df_bridges_all[
    (df_bridges_all["Kraj"] == "Košice") &
    (df_bridges_all["n_2023"] >= 6) &
    (df_bridges_all["Material"] == "prefabrikovaný predpätý betón") &
    ((df_bridges_all["Trieda_PK"] == "cesta II. triedy") |
     (df_bridges_all["Trieda_PK"] == "cesta III. triedy"))
]

filtered_bridges2.shape

(45, 51)

In [61]:
filtered_bridges2 = df_bridges_all[
    (df_bridges_all["Kraj"] == "Košice") &
    (df_bridges_all["n_2023"] >= 6) &
    (df_bridges_all["Material"] == "prefabrikovaný predpätý betón") &
    (df_bridges_all["Trieda_PK"] == "cesta I. triedy")
]

filtered_bridges2.shape

(3, 51)

In [63]:
bridges_specific = ["M1683"]

# Filter the DataFrame to include only bridges in Nitra
filtered_bridges_specific = df_bridges_all[df_bridges_all['ID_mosta'].isin(
    bridges_specific)]

filtered_bridges_specific.iloc[0]

ID_mosta                                                                           M1683
Trieda_PK                                                               cesta II. triedy
Číslo_PK                                                                             552
Most_DC                                                                  Dilatačný celok
Názov_mosta                                   Most cez inundačný kanál pred obcou Vojany
ID_DC                                                                           M1683.01
správcovské_číslo                                                                     32
Zemepisná_dĺžka                                                                21.973822
Zemepisná_šírka                                                                48.570076
Výška                                                                         106.095749
Druh_konštrukcie                                                                 trámová
Material             